In [27]:
from random import choice
import os

PATH = '/home/gemelli/projects/doc2graph/DATA/PAU/train'

Pick random image and annotations

In [28]:
image = choice([f for f in os.listdir(PATH) if f.endswith('tif')])
img_name = image.split('.')[0]
file_gt = img_name + '_gt.xml'
file_ocr = img_name + '_ocr.xml'

image, file_gt, file_ocr

('83565418.tif', '83565418_gt.xml', '83565418_ocr.xml')

In [29]:
import xml.etree.ElementTree as ET
root = ET.parse(os.path.join(PATH, file_gt)).getroot()
regions = []
for parent in root:
    if parent.tag.split("}")[1] == 'Page':
        for child in parent:
            region_label = child[0].attrib['value'] 
            if region_label == 'positions': region_label = 'table'
            region_bbox = [int(child[1].attrib['points'].split(" ")[0].split(",")[0].split(".")[0]),
                           int(child[1].attrib['points'].split(" ")[1].split(",")[1].split(".")[0]),
                           int(child[1].attrib['points'].split(" ")[2].split(",")[0].split(".")[0]),
                           int(child[1].attrib['points'].split(" ")[3].split(",")[1].split(".")[0])]
            regions.append([region_label, region_bbox])
            print([region_label, region_bbox])

['other', [21, 28, 728, 188]]
['invoice_info', [418, 192, 727, 272]]
['receiver', [21, 191, 411, 304]]
['supplier', [20, 307, 725, 417]]
['table', [21, 423, 726, 602]]
['total', [21, 607, 725, 746]]
['other', [20, 752, 726, 919]]


In [30]:
root = ET.parse(os.path.join(PATH, file_ocr)).getroot()
tokens = []
for parent in root:
    if parent.tag.split("}")[1] == 'Page':
        for child in parent:
            if child.tag.split("}")[1] == 'TextRegion':
                for elem in child:
                    if elem.tag.split("}")[1] == 'TextLine':
                        for word in elem:
                            if word.tag.split("}")[1] == 'Word':
                                word_bbox = [int(word[0].attrib['points'].split(" ")[0].split(",")[0].split(".")[0]),
                                             int(word[0].attrib['points'].split(" ")[1].split(",")[1].split(".")[0]),
                                             int(word[0].attrib['points'].split(" ")[2].split(",")[0].split(".")[0]),
                                             int(word[0].attrib['points'].split(" ")[3].split(",")[1].split(".")[0])]
                                word_text = word[1][0].text
                                center = lambda rect: ((rect[2]+rect[0])/2, (rect[3]+rect[1])/2)
                                c = center(word_bbox)
                                for reg in regions:
                                    r = reg[1]
                                    if r[0] < c[0] < r[2] and r[1] < c[1] < r[3]:
                                        word_label = reg[0]
                                        break
                                tokens.append([word_label, word_text, word_bbox])
                                print([word_text, word_bbox])

['Invoice', [99, 163, 153, 175]]
['DECEIVED', [556, 74, 661, 91]]
['DEC', [558, 104, 585, 118]]
['^', [606, 111, 613, 119]]
['1998', [618, 107, 646, 122]]
['DJM', [589, 143, 619, 157]]
['To:', [99, 199, 117, 209]]
['November', [538, 198, 597, 208]]
['18,', [603, 198, 618, 210]]
['1998', [625, 198, 650, 208]]
['Attorney', [99, 216, 148, 228]]
['David', [152, 215, 185, 226]]
['MacDougall', [189, 215, 257, 229]]
['Brennan,', [98, 232, 148, 244]]
['Steil,', [153, 232, 180, 244]]
['Basting', [185, 232, 226, 245]]
['ane', [230, 235, 249, 242]]
['MacDougall', [253, 232, 323, 245]]
['PO', [99, 249, 116, 259]]
['Box', [120, 249, 143, 259]]
['1148', [149, 249, 175, 259]]
['Janesville,', [98, 265, 154, 277]]
['Wisconsin', [159, 265, 215, 275]]
['53547', [220, 265, 254, 275]]
['From:', [98, 315, 131, 325]]
['Patricia', [98, 331, 141, 342]]
['J.', [145, 331, 152, 342]]
['O’Campo', [158, 331, 210, 345]]
['Ph', [215, 331, 229, 342]]
['D', [233, 332, 243, 341]]
['563-23-7979', [99, 348, 171, 358]]
['8

In [31]:
COLORS = {'invoice_info': (150, 75, 0), 'receiver':(0,100,0), 'other':(128, 128, 128), 'supplier': (255, 0, 255), 'table':(255,140,0), 'total':(0, 255, 255)}
def get_color(text):
    return COLORS[text]

In [32]:
from turtle import color
from PIL import Image, ImageDraw, ImageFont

img = Image.open(os.path.join(PATH, image)).convert('RGB')
draw = ImageDraw.Draw(img)
for rect in tokens:
    draw.rectangle(rect[2], outline=get_color(rect[0]), width=3)
for rect in regions:
    draw.rectangle(rect[1], outline=get_color(rect[0]), width=3)
    draw.text((rect[1][0], rect[1][1]-15), rect[0], font=ImageFont.truetype("Pillow/Tests/fonts/FreeMono.ttf", 15), fill='black')
img.save('pau.png')